# PM6

October 17th, 2020<br>Hannah Kim

In [1]:
%store -r publishersT
%store -r allArticles

Import necessary modules.

In [2]:
# import
from konlpy.tag import Okt
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt

Examine the format of the data.

In [3]:
allArticles[2]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'link': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=032&aid=0003030253',
 'title': '하태경 “의사-간호사 이간질 택한 문 대통령”',
 'datetime': datetime.datetime(2020, 9, 2, 17, 34),
 'num_comments': 150,
 'content': '하태경 국민의힘 의원이 지난달 26일 국회 회의를 주재하고 있다.   하태경 국민의힘(옛 미래통합당) 의원이 2일 문재인 대통령의 ‘간호사 격려’ 글에 “의사와 간호사 이간질시키는 문재인 대통령은 대통령이기를 포기하신 건가”라고 말했다  하 의원은 자신의 사회관계망서비스에서 이날 문 대통령의 메시지를 두고 “국민 이간질 해도해도 너무한다”면서 “의사와 간호사의 패싸움 하는 걸 조장하고 있다”고 말했다  그는 이어 “아무리 의사파업 중이라 해도 대통령이라면 절대 해서는 안될 행동”이라며 “해결책을 제시하지 못할망정 고생하는 간호사들을 부추겨 의사와 대결구도를 만들고 있으니 대통령이기를 포기하신 건지 되묻고 싶다”고 말했다  하 의원은 “문 대통령의 게시글을 본 국민 상당수가 두 눈을 의심할 정도”라며 “지금 대통령이 할 일은 더불어민주당이 약속한 의대증원 원점 재논의 명문화를 지시해 의사들을 즉각 병원에 복귀시키는 것”이라고 말했다  문 대통령은 이날 사회관계망서비스에 “전공의 등 의사들이 떠난 의료현장을 묵묵히 지키고 있는 간호사분들을 위로하며 헌신과 노고에 깊은 감사와 존경의 마음을 드린다”고 밝혔다  김은혜 국민의힘 대변인도 구두 논평을 통해 “(문 대통령의 메시지가) 의료진으로 표현되지만 대부분이 간호사였다. 대통령이 국민을 상대로 좌표를 찍었다”면서 “의사를 향한 대리전을 간호사들에 명하신 건가. 다음은 누구를 적으로 돌릴 셈인가”라고 말했다',
 'reactions': {'good': 344, 'warm': 4, 'sad': 1, 'angry': 45, 'w

Make a deep copy of allArticles.

In [4]:
allArticles_new = []
for d in allArticles:
    copy = {key: value for key, value in d.items()}
    allArticles_new.append(copy)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Use the Okt class in KoNLPy to split the content morphologically and attach them again with spaces in between.

In [ ]:
okt = Okt()

for a in allArticles_new:
    content = ' '.join([x.strip() for x in a['content'].split()])
    content = okt.morphs(content)
    content = [x for x in content if x not in "“”,\"()‘’·'…[]...-?"]
    a['content'] = ' '.join(content)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Examine data to see if there are more components to be cleaned up.

In [ ]:
content_counter = Counter()
for a in allArticles_new:
    words = [x.strip() for x in a['content'].split()]
    content_counter.update(words)
content_counter.most_common(100)

## Topic Modeling on Content

Make a list that only contains strings of article content.

In [ ]:
# all article content with new version
allContent_new = [x['content'] for x in allArticles_new]

TF vectorizer for content.

In [ ]:
# TF vectorizer
tf_vectorizer_content = CountVectorizer(strip_accents = 'unicode',
                                min_df = 0.05,
                                max_df = 0.85,
                                ngram_range = (2,5))

dtm_tf_content = tf_vectorizer_content.fit_transform(allContent_new)
print(dtm_tf_content.shape)

In [ ]:
print(tf_vectorizer_content.get_feature_names()[:50])

In [ ]:
print(tf_vectorizer_content.get_feature_names()[dtm_tf_content.shape[1]//2:dtm_tf_content.shape[1]//2+50])

In [ ]:
print(tf_vectorizer_content.get_feature_names()[-50:])

TF-IDF vectorizer for content.

In [ ]:
# TF-IDF vectorizer
tfidf_vectorizer_content = TfidfVectorizer(**tf_vectorizer_content.get_params())
dtm_tfidf_content = tfidf_vectorizer_content.fit_transform(allContent_new)
print(dtm_tfidf_content.shape)

Fit both LDA models for content.

In [ ]:
# Fit LDA Models

# TF DTM
lda_tf_content = LatentDirichletAllocation(n_components=5, random_state=0)
lda_tf_content.fit(dtm_tf_content)

# TF-IDF DTM
lda_tfidf_content = LatentDirichletAllocation(n_components=5, random_state=0)
lda_tfidf_content.fit(dtm_tfidf_content)

In [ ]:
# TF Model
pyLDAvis.sklearn.prepare(lda_tf_content, dtm_tf_content, tf_vectorizer_content)

In [ ]:
# TF-IDF Model
pyLDAvis.sklearn.prepare(lda_tfidf_content, dtm_tfidf_content, tfidf_vectorizer_content)

## Topic Modeling on Titles

Make a list that only contains the refined titles.

In [ ]:
allTitles_new = []

for a in allArticles_new:
    title = ' '.join([x.strip() for x in a['title'].split()])
    title = okt.morphs(title)
    title = [x for x in title if x not in "“”,\"()‘’·'…[]...-?"]
    allTitles_new.append(' '.join(title))

Examine data for further improvements in cleaning.

In [ ]:
title_counter = Counter()
for t in allTitles_new:
    words = [x.strip() for x in t.split()]
    title_counter.update(words)
title_counter.most_common(100)

Make TF vectorizer for titles.

In [ ]:
# TF vectorizer
tf_vectorizer_title = CountVectorizer(strip_accents = 'unicode',
                                min_df = 0.0015,
                                max_df = 0.005,
                                ngram_range = (2,5))

dtm_tf_title = tf_vectorizer_title.fit_transform(allTitles_new)
print(dtm_tf_title.shape)

In [ ]:
print(tf_vectorizer_title.get_feature_names()[:50])

In [ ]:
print(tf_vectorizer_title.get_feature_names()[dtm_tf_title.shape[1]//2:dtm_tf_title.shape[1]//2+50])

In [ ]:
print(tf_vectorizer_title.get_feature_names()[-50:])

Make TF-IDF vectorizer for titles.

In [ ]:
# TF-IDF vectorizer
tfidf_vectorizer_title = TfidfVectorizer(**tf_vectorizer_title.get_params())
dtm_tfidf_title = tfidf_vectorizer_title.fit_transform(allTitles_new)
print(dtm_tfidf_title.shape)

Fit both LDA models for article titles.

In [ ]:
# Fit LDA Models

# TF DTM
lda_tf_title = LatentDirichletAllocation(n_components=5, random_state=0)
lda_tf_title.fit(dtm_tf_title)

# TF-IDF DTM
lda_tfidf_title = LatentDirichletAllocation(n_components=5, random_state=0)
lda_tfidf_title.fit(dtm_tfidf_title)

In [ ]:
# TF Model
pyLDAvis.sklearn.prepare(lda_tf_title, dtm_tf_title, tf_vectorizer_title)

In [ ]:
# TF-IDF Model
pyLDAvis.sklearn.prepare(lda_tfidf_title, dtm_tfidf_title, tfidf_vectorizer_title)

## Analyze Topic Modeling Results

Dictionary for translating.

In [ ]:
translate = {
    '경향신문': 'Kyunghyang Shinmun',
    '국민일보': 'Kukmin Ilbo',
    '동아일보': 'DongA Ilbo',
    '문화일보': 'Munhwa Ilbo',
    '서울신문': 'Seoul Shinmun',
    '세계일보': 'Segye Ilbo',
    '조선일보': 'Chosun Ilbo',
    '중앙일보': 'JoongAng Ilbo',
    '한겨레': 'Hangyoreh',
    '한국일보': 'Hankuk Ilbo'
}

A function for getting lists of topic proportions for each publisher.

In [ ]:
def distrTopics(pList, publisher):
    """
    pList is the list of probabilities for an LDA model.
    publisher is the target publisher in which we are interested.
    distrTopics() will return a list where each index corresponds to each topic
    and each value is the number of articles for that topic.
    This function is only valid for models with 5 topics.
    The order of the items in pList must align with that of allArticles.
    """
    
    result = [0, 0, 0, 0, 0]
    for i in range(len(allArticles)):
        aDict = allArticles[i]
        if aDict['publisher'] == publisher:
            # Get topic (index) with highest probability
            maxP = 0
            maxIndex = 0
            for j in range(len(pList[i])):
                maxP = max(maxP, pList[i][j])
                if maxP == pList[i][j]:
                    maxIndex = j
            result[maxIndex] += 1
    
    total = sum(result)
    result = [x/total for x in result]
    return result

In [ ]:
probabilities_tfidf_content = lda_tfidf_content.transform(dtm_tfidf_content)
probabilities_tf_content = lda_tf_content.transform(dtm_tf_content)
probabilities_tfidf_title = lda_tfidf_title.transform(dtm_tfidf_title)
probabilities_tf_title = lda_tf_title.transform(dtm_tf_title)

topicDistr_tfidf_content = []
topicDistr_tf_content = []
topicDistr_tfidf_title = []
topicDistr_tf_title = []


for pbls in publishersT:
    topicDistr_tfidf_content.append(distrTopics(probabilities_tfidf_content, pbls))
    topicDistr_tf_content.append(distrTopics(probabilities_tf_content, pbls))
    topicDistr_tfidf_title.append(distrTopics(probabilities_tfidf_title, pbls))
    topicDistr_tf_title.append(distrTopics(probabilities_tf_title, pbls))

A function for producing the barplots of topic proportions for each publisher.

In [ ]:
def barDistr(topicDistrList, xt, yt, unit, publisher, basis):
    """
    topicDistrList is a list of proportions for each topic for each publisher.
    xt is a list of ticks for the x axis.
    yt is a list of ticks for the y axis.
    unit is a string for display in the title and y axis.
    publisher is the publisher in which we are interested.
    basis is TF or TF-IDF, whichever the model was based on.
    barDistr will display a plot of proportions of topics 
    for the articles by the publisher.
    The publisher must be one that is in the list publishersT.
    """
    
    pIndex = publishersT.index(publisher)
    
    plt.figure()
    plt.subplot()
    plt.bar(xt, topicDistrList[pIndex])
    plt.title('Proportions of {} Topics for {} Based on {}'.format(unit, translate[publisher], basis))
    plt.xlabel('Topics')
    plt.ylabel('Number of {}s'.format(unit))
    plt.xticks(xt)
    plt.yticks(yt)
    for i in range(5):
        val = topicDistrList[pIndex][i]
        plt.text(xt[i]-0.25, val+0.01, "{}%".format(round(val*100, 1)))
    plt.show()

In [ ]:
for p in publishersT:
    barDistr(topicDistr_tfidf_content, range(1,6), [0.1*x for x in range(0,11,2)], 'Content', p, 'TF-IDF')

In [ ]:
for p in publishersT:
    barDistr(topicDistr_tf_content, range(1,6), [0.1*x for x in range(0,11,2)], 'Content', p, 'TF')

In [ ]:
for p in publishersT:
    barDistr(topicDistr_tfidf_title, range(1,6), [0.1*x for x in range(0,11,2)], 'Title', p, 'TF-IDF')

In [ ]:
for p in publishersT:
    barDistr(topicDistr_tf_title, range(1,6), [0.1*x for x in range(0,11,2)], 'Title', p, 'TF')